In [1]:
stones = "92 0 286041 8034 34394 795 8 2051489"
stones_list = stones.split()

## Part 1

In [2]:
# Evolve the stones over 25 blinks
for i in range(25):

    new_stones = []
    for stone in stones_list:
        
        if stone == "0":
            new_stones.append("1")
            
        elif len(stone) % 2 == 0:
            mid = len(stone) // 2
            left = stone[:mid].lstrip("0") or "0"
            right = stone[mid:].lstrip("0") or "0"
            new_stones.extend([left, right])
            
        else:
            new_stones.append(str(int(stone) * 2024))
            
    stones_list = new_stones

print(len(stones_list), stones_list[:20])

239714 ['3', '2', '7', '7', '2', '6', '16192', '4048', '1', '4048', '8096', '3', '6', '8', '6', '9', '1', '8', '4', '2']


## Part 2

#### Caching Attempt

In [1]:
import os
import math
import functools
import pickle
import time
from collections import deque

In [2]:
# Path to stone and cache files
CACHE_FILE = "Data/stone_cache.pkl"
STATE_FILE = "Data/stone_state.pkl"

In [3]:
initial_stones = "92 0 286041 8034 34394 795 8 2051489"
initial_stones_list = list(map(int, initial_stones.split()))

In [4]:
@functools.lru_cache()
def count_digits(n):
    """ Returns the number of digits in a number. """
    if n == 0: return 1
    return math.floor(math.log10(abs(n))) + 1

In [5]:
@functools.lru_cache()
def split_stone(stone):
    """
    Splits a stone into two if it has an even number of digits.
    """
    num_digits = count_digits(stone)
            
    # Convert to string for splitting
    mid = num_digits // 2  # Determine middle index
                    
    # Split into left and right parts
    left = stone // (10 ** mid)
    right = stone % (10 ** mid)
                    
    return [left, right]

In [6]:
def evolve_stone(stone, cache):
    """
    Evolves a single stone based on the rules, with caching.
    Rule 1: Stone `0` becomes `1`.
    Rule 2: Stones with even digits are split into two.
    Rule 3: Other stones are multiplied by 2024.
    """
    if stone in cache: return cache[stone]
            
    # Compute transformation
    if stone != 0 and count_digits(stone) % 2 == 0:
        result = split_stone(stone)
            
    else:  # Rule 3
        result = [stone * 2024]
            
    # Store in cache
    cache[stone] = result
    return result

In [7]:
def save_state(stones_integers, blink, cache):
    state = {"stones": stones_integers, "blink": blink}
    
    with open(STATE_FILE, 'wb') as f: pickle.dump(state, f)
    with open(CACHE_FILE, 'wb') as f: pickle.dump(cache, f)

In [8]:
# Initialize the cache adhering to Rule 1
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "rb") as f:
        cache = pickle.load(f)
else:
    cache = {0: [1]}

In [10]:
# Load state if it exists, otherwise initialize
if os.path.exists(STATE_FILE):
    with open(STATE_FILE, "rb") as f:
        state = pickle.load(f)
        stones_integers = state["stones"]
        start_blink = state["blink"]
        
else:
    stones_integers = initial_stones_list
    start_blink = 0
        

In [11]:
save_step = 20
total_blinks = 75
save_interval = 300
last_save_time = time.time()

In [14]:
try:
    # Evaluate 75 blinks, saving every 5 minutes
    for blink in range(start_blink, 50, save_step):
        for step in range(save_step):
            
            if blink + step >= 50: break
            
            new_stones = deque()
            for i, stone in enumerate(stones_integers):
                evolved = evolve_stone(stone, cache)
                new_stones.extend(evolved)
                
                if i % 1000 == 0:  # Save every 1000 stones processed
                    stones_integers = list(new_stones)
                    save_state(stones_integers, blink, cache)
                    new_stones.clear()
                
            stones_integers = list(new_stones)
            print(f"Blink {blink + step + 1}: {len(stones_integers)} stones")
            
            # Save every 5 minutes
            current_time = time.time()
            if current_time - last_save_time >= 300:
                
                # Save current state
                save_state(stones_integers, blink + 1, cache)
                last_save_time = current_time
                
except MemoryError:
    print("Memory error encountered. Saving current progress...")
    save_state(stones_integers, blink, cache)

except KeyboardInterrupt:
    print("Processing stopped. Saving current progress...")
    save_state(stones_integers, blink, cache)
    

Blink 41: 191820485 stones
Blink 42: 290860273 stones
Blink 43: 442478291 stones
Blink 44: 671581769 stones
Blink 45: 1020067167 stones
Blink 46: 1550554346 stones
Blink 47: 2352405527 stones
Blink 48: 3576521900 stones


: 

: 

In [ ]:
# Final save after all blinks have elapsed
save_state(stones_integers, blink, cache)